In [264]:
# Libraries and and global variables
import numpy as np
import matplotlib.pyplot as plt
from cvxopt import matrix, solvers

K = None

In [265]:
from sklearn import datasets
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
#Loading dataset for classification, scale and split test and train
iris = datasets.load_iris()
# Filter and conserve tipe 0 and 2 of iris
cond = (iris.target != 1)
# Preprocess dataset to be used in SVM
y = iris.target[cond] - 1
y = y.astype('float')
x = iris.data[cond].astype('float')
x = preprocessing.scale(x)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25)


In [266]:
sigma = 1

# Custom rbf kernel
def krbf(x,y,sigma):
    return np.exp(-np.linalg.norm(x - y)**2/(2*sigma**2))

# SVM with rbf kernel
def fitNonLinear(x, y): 
    global K
    NUM = x.shape[0]
    DIM = x.shape[1]
    
    Y = np.reshape(y, (NUM, 1))
    Ym = np.matmul(Y,Y.T)
    # we'll solve the dual
    # obtain the kernel
    K = np.zeros((NUM,NUM))
    for i in range(NUM):
        for j in range(NUM):
            K[i,j] = krbf(x[i], x[j], sigma)
    K = np.multiply(K, Ym)
    P = matrix(K)
    q = matrix(-np.ones((NUM, 1)))
    G = matrix(-np.eye(NUM))
    h = matrix(np.zeros(NUM))
    A = matrix(y.reshape(1, -1))
    b = matrix(np.zeros(1))
    solvers.options['show_progress'] = False
    
    # Call to quadratic solver library
    sol = solvers.qp(P, q, G, h, A, b)
    alphas = np.array(sol['x'])
    return alphas

In [267]:
# fit svm classifier
alphas = fitNonLinear(X_train, y_train)

# Get support vectors and b
cond = (alphas > 1e-4)
alpha_sv = alphas[cond]
n_sv = len(alpha_sv)
xCond = cond.reshape(-1)
x_sv = X_train[xCond, :]
y_sv = y_train[xCond]
K_sv = K[xCond,xCond]

b = np.mean(y_sv - K_sv*(np.multiply(alpha_sv, y_sv)))


In [268]:
ym = np.zeros(X_test.shape[0])
# Predict the test data
Ktest2 = []
for j in range(X_test.shape[0]):
    x_p = X_test[j]
    K_pred=np.zeros(x_sv.shape[0]);
    for i in range(x_sv.shape[0]):
        K_pred[i] = krbf(x_p, x_sv[i], sigma)
    Ktest2.append(K_pred)
    ym[j] = np.sign(np.sum(np.multiply(np.multiply(alpha_sv,y_sv), K_pred)) + b)

In [269]:
from sklearn.metrics import confusion_matrix
#Create a confusion matrix to check the accuracy of the algorithm
confusion_matrix(y_test, ym)

array([[12,  0],
       [ 0, 13]], dtype=int64)

In [270]:
sigma = .4

# least square SVM with rbf kernel
def fitLS(x, y): 
    global K
    NUM = x.shape[0]
    DIM = x.shape[1]
    
    Y = np.reshape(y, (NUM, 1))
    Ym = np.matmul(Y,Y.T)
    # we'll solve the dual
    # obtain the kernel
    K = np.zeros((NUM,NUM))
    for i in range(NUM):
        for j in range(NUM):
            K[i,j] = krbf(x[i], x[j], sigma)

    Omega = np.multiply(K, Ym)
    onev = np.ones((NUM, 1))
    gamma = 1
    
    yforA = np.reshape(y, (NUM, 1))
    
    A11 = np.zeros(1).reshape((1,1))
    A12 = yforA.T
    A21 = yforA
    A22 = Omega + np.eye(NUM)/gamma
    
    A1 = np.hstack((A11, A12))
    A2 = np.hstack((A21, A22))

    A =  np.vstack((A1, A2))
    
    B = np.vstack((A11, onev))
    
    # Solve the linear system of equations
    sol = np.linalg.solve(A, B)
    b= sol[0]
    alpha=sol[1:]
    
    return {
        'b': b,
        'alphas': alpha
    }

In [271]:
# fit svm classifier
alphas = fitLS(X_train, y_train)
b = alphas['b']
alphas = alphas['alphas']

In [272]:
ym = np.zeros(X_test.shape[0])
# Predict the test data
for j in range(X_test.shape[0]):
    x_p = X_test[j]
    K_pred=np.zeros(X_train.shape[0]);
    for i in range(X_train.shape[0]):
        K_pred[i] = krbf(x_p, X_train[i], sigma)
    ym[j] = np.sign(np.sum(np.multiply(np.multiply(alphas,y_train), K_pred)) + b)

#Create a confusion matrix to check the accuracy of the algorithm
confusion_matrix(y_test, ym)

array([[12,  0],
       [ 0, 13]], dtype=int64)

In [279]:
sigma = 15

def fitLS_SVR(x, y): 
    global K
    NUM = x.shape[0]
    DIM = x.shape[1]
    
    Y = np.reshape(y, (NUM, 1))
    Ym = np.matmul(Y,Y.T)

    # we'll solve the dual
    # obtain the kernel
    K = np.zeros((NUM,NUM))
    for i in range(NUM):
        for j in range(NUM):
            K[i,j] = krbf(x[i], x[j], sigma)

    Omega = K
    onev = np.ones((NUM, 1))
    gamma = 15
    
    yforA = np.reshape(y, (NUM, 1))

    A11 = np.zeros(1).reshape((1,1))
    A12 = onev.T
    A21 = onev
    A22 = Omega + np.eye(NUM)/gamma
    
    A1 = np.hstack((A11, A12))
    A2 = np.hstack((A21, A22))

    A =  np.vstack((A1, A2))
    
    B = np.vstack((A11, yforA))
    
    # Solve the linear system of equations
    sol = np.linalg.solve(A, B)
    b= sol[0]
    alpha=sol[1:]
    
    return {
        'b': b,
        'alphas': alpha
    }

In [280]:
X, y = datasets.load_boston(return_X_y=True)
x = preprocessing.scale(X)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

# fit svm classifier
alphas = fitLS_SVR(X_train, y_train)

In [281]:
b = alphas['b']
alphas = alphas['alphas']

In [282]:
ym = np.zeros((y_test.shape[0]))

for j in range(y_test.shape[0]):
    x_p = X_test[j]
    K_pred=np.zeros((X_train.shape[0],1));
    for i in range(X_train.shape[0]):
        K_pred[i] = krbf(x_p, X_train[i], sigma)
    ym[j] = np.sum(np.multiply(alphas, K_pred)) + b

In [283]:
#from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
r2_score(y_test, ym)

0.7439106588663464

In [278]:
import pandas as pd

dataframe = pd.DataFrame(np.array([y_test, ym]).T)

dataframe.tail(30)

,0,1
97,23.2,25.556329
98,23.9,26.984053
99,50.0,37.993176
100,20.8,17.073516
101,21.7,22.011027
102,32.0,32.801937
103,21.2,22.601385
104,32.9,32.419526
105,19.5,20.214548
106,31.6,32.829239
